### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [9]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Source file: C:\Users\ping\Downloads\df_finviz_2025-12-26_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-12-26_df_finviz_stocks_etfs.parquet
Processing for date: 2025-12-26


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [ ]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine="pyarrow")
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-12-26_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: float64(31), int64(2), object(78)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,901,JAZZ,Jazz Pharmaceuticals plc,-,Healthcare,Biotechnology,Ireland,NASD,10.36B,NaN,7.59,1.24,2.49,2.62,5.06,7.75,65.18,33.67,-,NaN,-,0.00%,-6.07,6.39,-62.09%,183.47%,-1.00%,6.11%,13.48%,6.74%,19.88%,-179.96%,4.14%,4.16B,-368.48M,36.72%,1.35%,60.74M,57.98M,95.46%,4.59%,-4.69%,100.47%,-1.85%,8.70%,4.86,5.04M,-3.12%,-9.06%,-4.42%,1.65,1.44,1.11,1.37,73.01%,12.39%,-8.86%,2.70%,-5.32%,34.24%,59.48%,37.21%,38.43%,0.27,4.45,1.88%,2.46%,1.32%,9.34%,33.88%,-6.84%,32.36%,-6.84%,78.53%,95.49 - 182.99,-12.45%,32684.62%,59.37,Nov 05/a,6/1/2007,Yes,Yes,2800.0,0.01%,-0.19%,1.33,1.04M,0.25,254691,214.13,170.80,170.47,170.93,169.22,170.48,-0.19%,-,-,-,NaN,-,NaN,-,-,-,-,-,-,-,-,-
1,902,BAH,Booz Allen Hamilton Holding Corp,-,Industrials,Consulting Services,USA,NYSE,10.36B,13.04,14.11,6.82,0.88,10.46,12.69,12.65,8.16,6.73,2.62%,2.2,11/14/2025,28.69%,6.55,1.29,-11.69%,7.91%,16.31%,2.07%,9.93%,-8.15%,-52.95%,3.05%,2.42%,11.71B,820.68M,-1.45%,-2.70%,122.03M,119.96M,98.30%,1.11%,1.80%,99.66%,0.50%,6.44%,3.00,7.73M,11.70%,74.72%,16.31%,1.76,1.76,4.05,4.18,21.00%,9.77%,7.01%,-0.72%,2.08%,-13.82%,-16.98%,-34.87%,-33.65%,0.34,2.54,1.88%,2.68%,-2.10%,-2.19%,-17.37%,-16.43%,7.77%,-41.89%,7.77%,79.23 - 146.95,-55.20%,1172.06%,45.80,Oct 24/b,11/17/2010,Yes,Yes,35800.0,0.27%,0.01%,3.12,2.57M,0.40,1017645,100.27,85.15,85.16,85.53,84.51,85.39,0.28%,-,-,-,NaN,-,NaN,-,-,-,-,-,-,-,-,-
2,903,MGM,MGM Resorts International,S&P 500,Consumer Cyclical,Resorts & Casinos,USA,NYSE,10.31B,236.53,15.16,4.32,0.60,3.83,4.83,7.36,9.83,7.80,-,NaN,12/8/2022,0.00%,0.16,0.61,-12.55%,9.71%,-9.17%,3.51%,5.97%,1.61%,-272.53%,-94.07%,0.05%,17.28B,66.38M,-36.41%,0.24%,272.21M,204.41M,75.09%,25.26%,1.17%,68.99%,-2.29%,10.99%,4.77,22.45M,0.16%,2.25%,0.20%,1.23,1.19,11.71,11.78,39.16%,7.90%,0.38%,2.47%,10.37%,5.84%,9.63%,9.89%,8.74%,1.47,0.90,1.69%,2.37%,3.22%,10.91%,11.04%,-1.34%,29.11%,-8.81%,48.93%,25.30 - 41.32,-62.51%,1981.77%,65.03,Oct 29/a,5/2/1988,Yes,Yes,78000.0,1.18%,0.40%,2.20,4.71M,0.55,2587333,42.06,37.09,37.24,37.74,37.19,37.68,1.59%,-,-,-,NaN,-,NaN,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [ ]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")

    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace("-", "")
    df["Info"] = df[INFO_COLS].apply(
        lambda row: ", ".join(filter(None, row.astype(str))), axis=1
    )
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df["MktCap AUM"] = df["Market Cap"].replace("-", "") + df["AUM"].replace("-", "")
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[["Ticker", "Info", "MktCap AUM"]].head(3))


--- Step 2: Engineering new features ---
Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,JAZZ,"Healthcare, Biotechnology",10.36B
1,BAH,"Industrials, Consulting Services",10.36B
2,MGM,"Consumer Cyclical, Resorts & Casinos",10.31B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [ ]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan

    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {"T": 1_000_000, "B": 1_000, "M": 1, "K": 0.001}
    suffix = value_str[-1]

    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan


if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"

    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,10360.0,NaN,4160.0,-368.48,60.74,57.98,5.04,1.04,NaN,NaN,NaN,10360.0
1,10360.0,NaN,11710.0,820.68,122.03,119.96,7.73,2.57,NaN,NaN,NaN,10360.0
2,10310.0,NaN,17280.0,66.38,272.21,204.41,22.45,4.71,NaN,NaN,NaN,10310.0


#### Part B: Convert Percentage Columns to Numeric

In [ ]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col
        for col in df.columns
        if df[col].dtype == "object" and df[col].str.endswith("%", na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace("%", ""), errors="coerce")
            new_name = f"{col} %" if "%" not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

#### Part C: Convert Other String-Based Numeric Columns

In [ ]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == "object":
            df[col] = pd.to_numeric(
                df[col].str.replace(",", "", regex=False), errors="coerce"
            )
            converted_count += 1

    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 0 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [ ]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")

    # 1. Sort by the primary metric in descending order
    df.sort_values(
        by="MktCap AUM, M", ascending=False, inplace=True, na_position="last"
    )
    print("Sorted DataFrame by 'MktCap AUM, M'.")

    # 2. Add a 'Rank' column based on the new sort order
    df["Rank"] = range(1, len(df) + 1)
    print("Added 'Rank' column.")

    # 3. Set 'Ticker' as the index
    if "Ticker" in df.columns:
        df.set_index("Ticker", inplace=True)
        print("Set 'Ticker' as the index.")

    print("\nFinal DataFrame structure:")
    display(df[["Rank", "Info", "MktCap AUM, M"]].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",4629880.0
AAPL,2,"Technology, Consumer Electronics",4039850.0
GOOGL,3,"Communication Services, Internet Content & Inf...",3790970.0
GOOG,4,"Communication Services, Internet Content & Inf...",3790970.0
MSFT,5,"Technology, Software - Infrastructure",3624840.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [ ]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)

        # Save the file
        df.to_parquet(DEST_PATH, engine="pyarrow", compression="zstd")
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine="pyarrow")
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))

    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-12-26_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,4629880.0,47.19,24.83,0.51,24.74,38.95,76.39,59.88,4.89,2.49,0.02,0.04,12/4/2025,1.16,4.04,1.51,56.40,64.10,91.83,48.51,64.24,62.49,67.23,58.53,65.22,187140.0,99200.0,3.22,3.81,24300.00,23350.00,96.09,3.89,-0.93,68.18,-0.06,1.13,1.40,264.29,77.15,107.36,77.27,4.47,3.71,0.08,0.09,70.05,58.84,53.01,9.41,7.15,6.93,20.78,36.41,41.88,2.32,5.72,2.23,2.49,5.09,2.40,19.75,-10.21,12.37,-10.21,119.96,86.62 - 212.19,-10.21,571489.97,58.68,Nov 19/a,1/22/1999,Yes,Yes,36000.0,0.34,0.68,1.33,189.2700,0.73,139041558,260.77,188.61,189.89,192.69,188.00,190.53,1.02,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",4629880.0,1
AAPL,2,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,4039850.0,36.65,29.93,2.85,9.71,54.78,73.86,40.90,4.99,3.70,0.39,1.03,11/10/2025,13.66,7.46,2.66,10.39,10.94,17.91,10.49,8.71,7.94,91.14,22.85,6.43,416160.0,112010.0,4.10,0.23,14770.00,14760.00,99.92,0.10,-2.33,64.88,-0.22,0.83,2.62,122.04,30.93,171.42,68.44,0.89,0.86,1.22,1.52,46.91,31.97,26.92,0.44,-1.29,7.02,35.97,7.10,9.18,1.09,4.48,1.24,1.55,-1.28,0.69,18.44,-5.27,11.53,-5.27,61.57,169.21 - 288.62,-5.27,429666.04,49.12,Oct 30/a,12/12/1980,Yes,Yes,166000.0,-0.26,0.11,2.08,46.6200,0.46,21433429,292.51,273.81,274.10,275.37,272.86,273.40,-0.15,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",4039850.0,2
GOOGL,4,Alphabet Inc,"NDX, S&P 500",Communication Services,Internet Content & Information,USA,NASD,3790970.0,30.93,27.90,1.63,9.83,9.79,38.49,51.54,32.03,8.15,0.17,0.83,12/8/2025,7.46,10.14,2.63,32.15,5.74,26.76,17.16,16.73,16.21,35.35,34.26,13.55,385520.0,124250.0,26.79,2.41,5820.00,5800.00,99.68,51.94,0.02,38.75,-0.70,1.42,2.27,82.15,25.71,35.45,29.40,1.75,1.75,0.09,0.11,59.18,33.10,32.23,3.65,-3.07,27.16,75.61,61.08,65.62,1.08,7.63,1.48,2.24,0.24,6.99,45.75,-4.66,28.41,-4.66,123.09,140.53 - 328.83,-4.66,12955.71,57.64,Oct 29/a,8/19/2004,Yes,Yes,183323.0,-0.32,0.14,1.43,36.1800,0.30,10864603,334.50,314.09,314.52,315.08,312.27,313.51,-0.18,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Communication Services, Internet Content & Inf...",3790970.0,3
GOOG,3,Alphabet Inc,"NDX, S&P 500",Communication Services,Internet Content & Information,USA,NASD,3790970.0,31.07,28.03,1.63,9.83,9.83,38.49,51.54,32.03,8.18,0.18,0.83,12/8/2025,7.46,10.14,2.63,32.15,5.76,26.76,17.16,16.73,16.21,35.35,34.26,13.55,385520.0,124250.0,26.62,2.40,5420.00,5020.00,92.69,58.39,0.01,26.72,-0.71,1.12,2.41,56.15,25.71,35.45,29.40,1.75,1.75,0.09,0.11,59.18,33.10,32.23,3.69,-2.68,27.42,76.68,60.70,65.39,1.09,7.59,1.50,2.22,0.39,7.23,45.70,-4.17,28.73,-4.17,120.78,142.66